In [2]:
# отключим всякие предупреждения Anaconda
import warnings

warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [3]:
data = pd.read_csv('./data/winequality-white.csv', sep=';')

In [4]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [8]:
X = np.array(data.iloc[:,[x for x in range(0, 11)]])
y = np.array(data["quality"])

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size = 0.7, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [9]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression()

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [45]:
from sklearn.metrics import mean_squared_error
train_pred = linreg.predict(X_train_scaled)
test_pred = linreg.predict(X_holdout_scaled)

print("Mean squared error (train): ", mean_squared_error(y_train, train_pred))
print("Mean squared error (test): ", mean_squared_error(y_holdout, test_pred))
print("================== with round ==================")
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, train_pred))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, test_pred))

Mean squared error (train):  0.5321779540981313
Mean squared error (test):  0.58351984586918
================== with round ==================
Mean squared error (train): 0.532
Mean squared error (test): 0.584


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [14]:
linreg_coef = pd.DataFrame(linreg.coef_, index= data.columns[:11])
linreg_coef.sort_values(0, ascending=False)
print(linreg_coef)

                             0
fixed acidity         0.100363
volatile acidity     -0.192491
citric acid           0.013249
residual sugar        0.560499
chlorides             0.011563
free sulfur dioxide  -0.010237
total sulfur dioxide  0.033827
density              -0.674179
pH                    0.151993
sulphates             0.065973
alcohol               0.140476


## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [16]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [17]:
lasso1_coef = pd.DataFrame(lasso1.coef_, index=data.columns[:11])
lasso1_coef.sort_values(0, ascending=False)

,0
alcohol,0.343034
residual sugar,0.251695
pH,0.063068
sulphates,0.032678
free sulfur dioxide,0.001119
fixed acidity,-0.000000
citric acid,0.000000
chlorides,-0.000000
total sulfur dioxide,0.000000
volatile acidity,-0.189060


**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [18]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        random_state=17)

In [19]:
lasso_cv.alpha_

0.0014992684327860455

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. **<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [20]:
lasso_cv_coef = pd.DataFrame(lasso1.coef_, index=data.columns[:11])
lasso_cv_coef.sort_values(0, ascending=False)

,0
alcohol,0.343034
residual sugar,0.251695
pH,0.063068
sulphates,0.032678
free sulfur dioxide,0.001119
fixed acidity,-0.000000
citric acid,0.000000
chlorides,-0.000000
total sulfur dioxide,0.000000
volatile acidity,-0.189060


**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [46]:
train_pred = lasso_cv.predict(X_train_scaled)
test_pred = lasso_cv.predict(X_holdout_scaled)

print("Mean squared error (train): ", mean_squared_error(y_train, train_pred))
print("Mean squared error (test): ", mean_squared_error(y_holdout, test_pred))
print("================== with round ==================")
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, train_pred))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, test_pred))


Mean squared error (train):  0.5324734177725564
Mean squared error (test):  0.5812632364411521
================== with round ==================
Mean squared error (train): 0.532
Mean squared error (test): 0.581
